#### В коде из методички реализуйте один или несколько критериев останова (количество листьев, признаков, глубина дерева и т.д.).

#### Реализуйте дерево для задачи регрессии. Возьмите за основу дерево, реализованное в методичке, заменив механизм предсказания в листе на взятие среднего значения по выборке, и критерий Джинни на дисперсию значений.